In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

chat = ChatOpenAI(
    temperature=0.1,
    ##streaming=True,
    ##callbacks=[
    ##    StreamingStdOutCallbackHandler(),
    ##],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "question": "Please explain the movie The Top Gun using 3 emojis",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "question": "Please explain the movie The Godfather using 3 emojis",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
]

example_prompt = ChatPromptTemplate.from_template("Human:{question}.\nAI:{answer}")

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are making something that summarizes a movie using 3 emojis."),
        example_prompt,MessagesPlaceholder(variable_name="history"),
        ("human", "  {question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [4]:
invoke_chain("Please explain the movie The Dark Knight using 3 emojis")

content='🦇🃏💥'


In [5]:
invoke_chain("Please explain the movie The Oppenheimer using 3 emojis")

content='🔬💣🌌'


In [6]:
invoke_chain("Please express the movie mentioned earlier again.")

content='I\'m sorry for the confusion. Here is the summary of the movie "The Oppenheimer" using 3 emojis:\n\n🔬💣🌌'


In [7]:
invoke_chain("Please express all movies again.")

content='Sure! Here are the summaries of the movies using 3 emojis each:\n\n1. The Top Gun: 🛩️👨\u200d✈️🔥\n2. The Godfather: 👨\u200d👨\u200d👦🔫🍝\n3. The Dark Knight: 🦇🃏💥\n4. The Oppenheimer: 🔬💣🌌'
